**Assignment 6**

**Name: Anuska Nath**

**IT (UG3), section:A3**

**Roll: 002311001003**

Look at the classical 8-puzzle problem and its solution from a particular given start state to a specific end state.

Write a Python program to solve the problem using **A*** **Search** in such a way that it can handle the 15-Puzzle problem with the same program.

Choose any suitable heuristic function and execute it in the problem.

Provide necessary inputs to the program. Do not statically mention any of them inside the program.

Input and output states shall be written in the same file and you can read them properly as required.

Intermediate output shall be properly displayed and will be written in a separate output file.

In [32]:
import heapq
import sys
from typing import List, Tuple

# Node structure for A* Search
class PuzzleNode:
    def __init__(self, state, parent=None, move=None, depth=0, cost=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost


# A* Solver class
class PuzzleSolver:
    def __init__(self, start: List[List[int]], goal: List[List[int]]):
        self.start = start
        self.goal = goal
        self.n = len(start)
        self.goal_pos = self.build_goal_position()

    def build_goal_position(self):
        position = {}
        for i in range(self.n):
            for j in range(self.n):
                position[self.goal[i][j]] = (i, j)
        return position

    def manhattan_distance(self, state):
        dist = 0
        for i in range(self.n):
            for j in range(self.n):
                val = state[i][j]
                if val != 0:
                    goal_i, goal_j = self.goal_pos[val]
                    dist += abs(goal_i - i) + abs(goal_j - j)
        return dist

    def get_neighbors(self, state):
        neighbors = []
        for i in range(self.n):
            for j in range(self.n):
                if state[i][j] == 0:
                    x, y = i, j
        directions = [('Up', (-1, 0)), ('Down', (1, 0)), ('Left', (0, -1)), ('Right', (0, 1))]
        for move, (dx, dy) in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.n and 0 <= ny < self.n:
                new_state = [row[:] for row in state]
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                neighbors.append((new_state, move))
        return neighbors

    def state_to_tuple(self, state):
        return tuple(tuple(row) for row in state)

    def reconstruct_path(self, node):
        path = []
        while node:
            path.append((node.move, node.state))
            node = node.parent
        path.reverse()
        return path

    def solve(self):
        open_set = []
        closed_set = set()
        start_node = PuzzleNode(self.start, None, None, 0, self.manhattan_distance(self.start))
        heapq.heappush(open_set, (start_node.cost, start_node))
        while open_set:
            _, current = heapq.heappop(open_set)
            if current.state == self.goal:
                return self.reconstruct_path(current)
            closed_set.add(self.state_to_tuple(current.state))
            for neighbor_state, move in self.get_neighbors(current.state):
                neighbor_tuple = self.state_to_tuple(neighbor_state)
                if neighbor_tuple in closed_set:
                    continue
                g = current.depth + 1
                h = self.manhattan_distance(neighbor_state)
                neighbor_node = PuzzleNode(neighbor_state, current, move, g, g + h)
                heapq.heappush(open_set, (neighbor_node.cost, neighbor_node))
        return None


def read_input_file(filename: str) -> Tuple[List[List[int]], List[List[int]]]:
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip() != '']  # skip empty lines
    n = int(lines[0])
    if len(lines) < 1 + 2 * n:
        raise ValueError(f"Expected at least {1 + 2 * n} lines, got {len(lines)}")
    start = [list(map(int, lines[i].split())) for i in range(1, 1 + n)]
    goal = [list(map(int, lines[i].split())) for i in range(1 + n, 1 + 2 * n)]
    return start, goal


def write_output(path: List[Tuple[str, List[List[int]]]], filename='output.txt'):
    with open(filename, 'w') as f:
        for i, (move, state) in enumerate(path):
            output = f"Step {i}: {'Initial' if move is None else move}\n"
            for row in state:
                output += ' '.join(map(str, row)) + '\n'
            output += '\n'
            print(output, end='')
            f.write(output)
        f.write(f"Total steps: {len(path) - 1}\n")
        print(f"Total steps: {len(path) - 1}")


if __name__ == '__main__':
    #running for input1.txt (8-puzzle)
    input_file = 'input1.txt'
    start_state, goal_state = read_input_file(input_file)
    solver = PuzzleSolver(start_state, goal_state)
    path = solver.solve()
    print()
    print("Output for input1.txt (8-puzzle):")
    print()
    if path:
        write_output(path, 'output1.txt')
    else:
        print("No solution found.")
        with open('output1.txt', 'w') as f:
            f.write("No solution found.\n")

    #running for input2.txt (15-puzzle)
    input_file = 'input2.txt'
    start_state, goal_state = read_input_file(input_file)
    solver = PuzzleSolver(start_state, goal_state)
    path = solver.solve()
    print()
    print("Output for input2.txt (15-puzzle):")
    print()
    if path:
        write_output(path, 'output2.txt')
    else:
        print("No solution found.")
        with open('output2.txt', 'w') as f:
            f.write("No solution found.\n")



Output for input1.txt (8-puzzle):

Step 0: Initial
1 2 3
4 0 5
6 7 8

Step 1: Right
1 2 3
4 5 0
6 7 8

Step 2: Down
1 2 3
4 5 8
6 7 0

Step 3: Left
1 2 3
4 5 8
6 0 7

Step 4: Left
1 2 3
4 5 8
0 6 7

Step 5: Up
1 2 3
0 5 8
4 6 7

Step 6: Right
1 2 3
5 0 8
4 6 7

Step 7: Down
1 2 3
5 6 8
4 0 7

Step 8: Right
1 2 3
5 6 8
4 7 0

Step 9: Up
1 2 3
5 6 0
4 7 8

Step 10: Left
1 2 3
5 0 6
4 7 8

Step 11: Left
1 2 3
0 5 6
4 7 8

Step 12: Down
1 2 3
4 5 6
0 7 8

Step 13: Right
1 2 3
4 5 6
7 0 8

Step 14: Right
1 2 3
4 5 6
7 8 0

Total steps: 14

Output for input2.txt (15-puzzle):

Step 0: Initial
1 0 2 4
5 7 3 8
9 6 11 12
13 10 14 15

Step 1: Right
1 2 0 4
5 7 3 8
9 6 11 12
13 10 14 15

Step 2: Down
1 2 3 4
5 7 0 8
9 6 11 12
13 10 14 15

Step 3: Left
1 2 3 4
5 0 7 8
9 6 11 12
13 10 14 15

Step 4: Down
1 2 3 4
5 6 7 8
9 0 11 12
13 10 14 15

Step 5: Down
1 2 3 4
5 6 7 8
9 10 11 12
13 0 14 15

Step 6: Right
1 2 3 4
5 6 7 8
9 10 11 12
13 14 0 15

Step 7: Right
1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 0

